In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import helpers.processing_helpers as ph
from sklearn.svm import LinearSVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
df_dev = pd.read_csv("./development.csv")

acc_idxs = [1,2,3,4,5,6,8,9,10,11,13,14]
noise_indexes = [0,7,12,15,16,17]



features = ['pmax', 'negpmax', 'area', 'tmax', 'rms']

drop_features = ['area', 'tmax', 'rms']

df_dev = df_dev.drop(columns=ph.get_column_names(features, noise_indexes)) # Drop noisy sensors
df_dev = df_dev.drop(columns=ph.get_column_names(drop_features, acc_idxs))


# std_threshold = 5
# means_by_point = df_dev.groupby(['x', 'y']).mean().reset_index()
# duplicate_means = means_by_point.loc[means_by_point.index.repeat(100)].reset_index(drop=True)
# stds_by_point = df_dev.groupby(['x', 'y']).std().reset_index()
# duplcate_stds = stds_by_point.loc[stds_by_point.index.repeat(100)].reset_index(drop=True)
# df_dev = df_dev[(np.abs((df_dev-duplicate_means) / duplcate_stds) < std_threshold).all(axis=1)]




# for value in acc_idxs:
#     feature = "diff[" + str(value) + "]"
#     pmax = "pmax[" + str(value) + "]"
#     negpmax = "negpmax[" + str(value) + "]"
#     df_dev[feature] = df_dev[pmax] - df_dev[negpmax]


y_train_valid = df_dev[['x', 'y']].copy()

X_train_valid = df_dev.drop(columns=['x', 'y'])

X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True)

X = pd.concat([y_train, X_train], axis=1)

means = X_train.mean()
stds = X_train.std()

# std_threshold = 10

# means_by_point = X.copy().groupby(['x', 'y']).mean().reset_index()
# duplicate_means = means_by_point.loc[means_by_point.index.repeat(100)].reset_index(drop=True)
# stds_by_point = X.copy().groupby(['x', 'y']).std().reset_index()
# duplcate_stds = stds_by_point.loc[stds_by_point.index.repeat(100)].reset_index(drop=True)
# X = X[(np.abs((X-duplicate_means) / duplcate_stds) < std_threshold).all(axis=1)]

# X_train = X.drop(columns=['x', 'y'])
# y_train = X[['x', 'y']]



X_train_normalized = (X_train - means) / stds

X_valid_normalized = (X_valid - means) / stds

In [ ]:
df_ev = pd.read_csv("./evaluation.csv")
features = ["pmax", "negpmax", 'area', 'tmax', 'rms']

eval_noise_removed = df_ev.drop(columns=ph.get_column_names(features, noise_indexes))
eval_noise_removed = eval_noise_removed.drop(columns=ph.get_column_names(drop_features, acc_idxs))

indices = eval_noise_removed["Id"].copy()
eval_noise_removed = eval_noise_removed.drop(columns = ["Id"])
eval_normalized = (eval_noise_removed - means) / stds


In [ ]:
mlp = MLPRegressor(random_state=42, verbose=1, n_iter_no_change=500, max_iter=200, learning_rate_init=0.01, activation="logistic", learning_rate="adaptive")
mlp.fit(X_train_normalized, y_train)

In [ ]:
y_pred = mlp.predict(X_valid_normalized)
print(ph.mean_euclid_dist(y_valid, y_pred))

In [ ]:
def round_to_nearest_5(n):
    return np.round(n / 5) * 5


rounding_vectorized = np.vectorize(round_to_nearest_5)

In [ ]:
y_pred_rounded = round_to_nearest_5(y_pred)
print(ph.mean_euclid_dist(y_valid, y_pred_rounded))

In [ ]:
last_run = 4.645704973901679

In [ ]:
print(y_valid)

In [ ]:
normalized_eval_tms_rms_area = (eval_noise_removed - means) / stds
y_pred_eval = mlp.predict(normalized_eval_tms_rms_area)


f = open("predictions/no_outliers_tmax_rms_area.csv", 'w')
f.write("Id,Predicted")
f.write("\n")

for ind in range(y_pred_eval.shape[0]):
    line = str(indices[ind]) + "," + str(y_pred_eval[ind,0]) + "|" + str(y_pred_eval[ind,1])
    f.write(line)
    f.write("\n")
print(ind)

In [ ]:
# acc_idxs = (1,2,3,4,5,6,8,9,10,11,13,14)
# X_train_normalized =  X_train_normalized.drop(columns=ph.get_column_names(['tmax', 'rms'], acc_idxs))
# X_valid_normalized = X_valid_normalized.drop(columns=ph.get_column_names(['tmax', 'rms'], acc_idxs))
# print(X_train_normalized.columns)

In [ ]:
# mlp_tmax_rms_dropped = MLPRegressor(random_state=42, verbose=1, n_iter_no_change=500, max_iter=1500, learning_rate_init=0.001, activation="logistic")
# mlp_tmax_rms_dropped.fit(X_train_normalized, y_train)

In [ ]:
# print(X_train_normalized)

In [ ]:
# y_pred_tmax_rms_dropped = mlp_tmax_rms_dropped.predict(X_valid_normalized)




# print(ph.mean_euclid_dist(y_valid, y_pred_tmax_rms_dropped))

In [ ]:
# normalized_eval_area = (eval_noise_removed - means) / stds

# print(eval_noise_removed.columns)

# normalized_eval_area =  eval_noise_removed.drop(columns=ph.get_column_names(['tmax', 'rms'], acc_idxs))

# y_pred_eval = mlp_tmax_rms_dropped.predict(normalized_eval_area)





# f = open("predictions/no_outliers_area_.csv", 'w')
# f.write("Id,Predicted")
# f.write("\n")

# for ind in range(y_pred_eval.shape[0]):
#     line = str(indices[ind]) + "," + str(y_pred_eval[ind,0]) + "|" + str(y_pred_eval[ind,1])
#     f.write(line)
#     f.write("\n")
# print(ind)

In [ ]:
# acc_idxs = (1,2,3,4,5,6,8,9,10,11,13,14)
# X_train_normalized =  X_train_normalized.drop(columns=ph.get_column_names(['area'], acc_idxs))
# X_valid_normalized = X_valid_normalized.drop(columns=ph.get_column_names(['area'], acc_idxs))

In [ ]:
# mlp_tmax_rms_area_dropped = MLPRegressor(random_state=42, verbose=1, n_iter_no_change=500, max_iter=1500, learning_rate_init=0.001, activation="logistic")
# mlp_tmax_rms_area_dropped.fit(X_train_normalized, y_train)

In [ ]:
# y_pred_tmax_rms_area_dropped = mlp_tmax_rms_area_dropped.predict(X_valid_normalized)
# print(ph.mean_euclid_dist(y_valid, y_pred_tmax_rms_dropped))

In [ ]:
# normalized_eval_area = (eval_noise_removed - means) / stds

# print(eval_noise_removed.columns)

# normalized_eval_area =  eval_noise_removed.drop(columns=ph.get_column_names(['tmax', 'rms', 'area'], acc_idxs))

# y_pred_eval = mlp_tmax_rms_area_dropped.predict(normalized_eval_area)

# f = open("predictions/no_outliers.csv", 'w')
# f.write("Id,Predicted")
# f.write("\n")

# for ind in range(y_pred_eval.shape[0]):
#     line = str(indices[ind]) + "," + str(y_pred_eval[ind,0]) + "|" + str(y_pred_eval[ind,1])
#     f.write(line)
#     f.write("\n")
# print(ind)

In [ ]:
# y_train_valid = pmax_negpmax_area[['x', 'y']].copy()

# X_train_valid = pmax_negpmax_area.drop(columns=['x', 'y'])

# X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

In [ ]:
# y_train_valid = pmax_negpmax_area[['x', 'y']].copy()

# area_idx = (1,2,3,4,5,6,8,9,10,11,13,14)
# X_train_valid = pmax_negpmax_area.drop(columns=['x', 'y']).drop(columns=ph.get_column_names(['area'], area_idx))

# X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42, train_size=0.9)

# print(X_valid.shape[0] / (X_train.shape[0] + X_valid.shape[0]))



In [ ]:
# means = X_train.mean()
# stds = X_train.std()
# X_train_normalized = (X_train - means) / stds

# print(X_train_normalized.shape)

In [ ]:
# mlp = MLPRegressor(random_state=42, verbose=1, n_iter_no_change=500, max_iter=1500, learning_rate_init=0.0001, activation="logistic")
# mlp.fit(X_train_normalized, y_train)

In [ ]:
# X_valid_normalized = (X_valid - means) / stds
# y_hat = mlp.predict(X_valid_normalized)
# mlp_med = ph.mean_euclid_dist(y_valid, y_hat)
# print(mlp_med)

In [ ]:
def round_to_nearest_5(n):
    return np.round(n / 5) * 5


rounding_vectorized = np.vectorize(round_to_nearest_5)

In [ ]:
# print(ph.mean_euclid_dist(y_valid, y_pred))
# print(ph.mean_euclid_dist(y_valid, y_pred_tmax_rms_dropped))
# print(ph.mean_euclid_dist(y_valid, y_pred_tmax_rms_area_dropped))
# print(ph.mean_euclid_dist(y_valid, round_to_nearest_5(y_pred)))
# print(ph.mean_euclid_dist(y_valid, round_to_nearest_5(y_pred_tmax_rms_dropped)))
# print(ph.mean_euclid_dist(y_valid, round_to_nearest_5(y_pred_tmax_rms_area_dropped)))

In [ ]:
# df_ev = pd.read_csv("./evaluation.csv")

# print(df_ev.shape)

In [ ]:
# noise_indexes = [0,7,12,15,16,17]
# features = ["pmax", "negpmax", 'area', 'tmax', 'rms']

# eval_noise_removed = df_ev.drop(columns=ph.get_column_names(features, noise_indexes))

# print(eval_noise_removed.shape)

In [ ]:
# indices = eval_noise_removed["Id"].copy()
# eval_noise_removed = eval_noise_removed.drop(columns = ["Id"])
# print(eval_noise_removed.shape)

In [ ]:
# acc_idxs = [1,2,3,4,5,6,8,9,10,11,13,14]
# eval_noise_removed = eval_noise_removed.drop(columns=ph.get_column_names(['tmax', 'rms'], acc_idxs))
# print(eval_noise_removed)



In [ ]:
# normalized_eval = (eval_noise_removed - means) / stds

In [ ]:
# y_pred_eval = mlp_tmax_rms_dropped.predict(normalized_eval)
# print(y_pred_eval.shape)

In [ ]:
# f = open("predictions/no_outliers_tmax_rms_area.csv", 'w')
# f.write("Id,Predicted")
# f.write("\n")

# for ind in range(y_pred_eval.shape[0]):
#     line = str(indices[ind]) + "," + str(y_pred_eval[ind,0]) + "|" + str(y_pred_eval[ind,1])
#     f.write(line)
#     f.write("\n")
# print(ind)

In [ ]:
# rf = RandomForestRegressor(n_estimators=200, random_state=42)
# rf.fit(X_train_normalized, y_train)

In [ ]:
# y_train_valid = pmax_negpmax_area[['x', 'y']].copy()

# X_train_valid = pmax_negpmax_area.drop(columns=['x', 'y'])

# X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

# noise_indexes = [0,7,12,15,16,17]
# features = ["pmax", "negpmax", 'area', 'tmax', 'rms']

# noise_removed = subset.drop(columns=ph.get_column_names(features, noise_indexes))




In [ ]:
# means_outliers_removed = X_train_outliers_removed.mean()
# stds_outliers_removed = X_train_outliers_removed.std()
# X_train_normalized = (X_train_outliers_removed - means) / stds

# print(X_train_normalized)

In [ ]:
# mlp = MLPRegressor(random_state=42, verbose=1, n_iter_no_change=500, max_iter=1500, learning_rate_init=0.001, activation="logistic")
# mlp.fit(X_train_normalized, y_train)

In [ ]:
# y_hat_outliers_removed = mlp.predict(X_valid_normalized)
# print(ph.mean_euclid_dist(y_valid, y_hat_outliers_removed))